<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/openai_agents_sdk/agentic_campaign_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
openai_api_key=userdata.get('OPENAI_API_KEY')
sendgrid_api_key=userdata.get('SENDGRID_API_KEY')

In [24]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["GOOGLE_API_KEY"] = google_api_key
os.environ["SENDGRID_API_KEY"] = sendgrid_api_key

In [3]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.6/734.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0


In [6]:
!pip install sendgrid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 10.6 MB/s eta 0:00:00


In [7]:
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [8]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [9]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [10]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution specifically designed to streamline SOC 2 compliance and audit preparations.

Navigating the complexities of SOC 2 requirements can be overwhelming, especially in today’s fast-paced business environment. ComplAI leverages advanced AI technology to automate compliance tasks, saving your team valuable time and reducing the risk of oversight. Our platform offers:

- **Automated Documentation**: Easily generate and manage compliance documentation.
- **Real-time Monitoring**: Stay up-to-date with compliance requirements and changes.
- **Seamless Audit Preparation**: Simplify the audit process with organized, accessible records.

We’ve helped organizations like yours not only achieve compliance but do so efficiently and confidently. I would love to discuss how ComplAI ca

In [11]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Secure Your SOC 2 Compliance with ComplAI

Hi [First Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in helping organizations like yours achieve SOC 2 compliance efficiently and effectively.

In today’s digital landscape, ensuring data security and compliance can be a complex challenge. Our AI-powered SaaS tool streamlines the entire compliance process, from risk assessments to audit preparation, saving your team valuable time and resources.

Key benefits of using ComplAI include:
- Automated documentation and policy generation
- Real-time compliance monitoring
- Tailored guidance based on your specific industry needs

I’d love to explore how we can help your organization simplify SOC 2 compliance. Would you be available for a brief call next week?

Thank you for your time, and I look forward to connecting.

Best regards,

[Your Name]  
[Your Position]  
ComplAI  
[Your Phone Number]  
[Your Email Addre

In [12]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [25]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Does Your Audit Preparation Make You Want to Pull Your Hair Out? 🧑‍🦲

Hey [First Name],

Ever feel like preparing for a SOC2 audit is like trying to find a unicorn in a haystack? 🦄 Well, I’m here to tell you that you don't have to go on a magical quest anymore!

At ComplAI, we believe that compliance should be less “head-scratching” and more “cruising on autopilot.” Our AI-powered tool helps you sail smoothly through the murky waters of SOC2 compliance, leaving you free to focus on what you do best—running your business (and maybe even taking a much-deserved coffee break!).

Imagine a world where audits are as easy as pie (the kind that doesn’t make you feel guilty afterward). With ComplAI, we’ll take care of the compliance mumbo-jumbo so you can skip the stress and still look good in front of your stakeholders.

Curious to learn how it works? Let’s schedule a quick chat and turn that audit anxiety into confidence! 🚀

Best  
[Your Name]  
Chief Compliance Che

#Using Tools

In [15]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)


In [16]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [17]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("data.savvy.learn@gmail.com")  # Change to your verified sender
    to_email = To("aforalgo@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [18]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ae80fb5eca0>, strict_json_schema=True, is_enabled=True)

In [19]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ae80e6a1120>, strict_json_schema=True, is_enabled=True)

In [20]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ae80e356c00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ae80e356b60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

In [26]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

ERROR:openai.agents:Error getting response: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-r2UWGvZfAxwY64LO4s2miWUO on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}. (request_id: req_298593dfc539a01297ef5e487e23594b)


In [27]:
print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to the CEO. If you need any more assistance or another outreach, feel free to ask!
- 9 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
